In [12]:
# Import relevant libraries
import pandas as pd
from pycaret.classification import *

In [13]:
# Import training dataset
train = pd.read_csv('/Users/zarahaziz/Documents/Data_Science/All_Projects/6_FraudDetection/data/train.csv', delimiter='|')
train.head()

,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition,fraud
0,5,1054,54.70,7,0,3,0.027514,0.051898,0.241379,0
1,3,108,27.36,5,2,4,0.129630,0.253333,0.357143,0
2,3,1516,62.16,3,10,5,0.008575,0.041003,0.230769,0
3,6,1791,92.31,8,4,4,0.016192,0.051541,0.275862,0
4,5,430,81.53,3,7,2,0.062791,0.189605,0.111111,0


In [14]:
# Import test dataset
test = pd.read_csv('/Users/zarahaziz/Documents/Data_Science/All_Projects/6_FraudDetection/data/test.csv', delimiter='|')
test.head()

,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition
0,4,467,88.48,4,8,4,0.014989,0.189465,0.571429
1,3,1004,58.99,7,6,1,0.026892,0.058755,0.259259
2,1,162,14.00,4,5,4,0.006173,0.086420,4.000000
3,5,532,84.79,9,3,4,0.026316,0.159380,0.642857
4,5,890,42.16,4,0,0,0.021348,0.047371,0.210526


In [15]:
# Use pycaret for classification algorithm
environment = setup(train, target='fraud', session_id=123)

,Description,Value
0,Session id,123
1,Target,fraud
2,Target type,Binary
3,Original data shape,"(1879, 10)"
4,Transformed data shape,"(1879, 10)"
5,Transformed train set shape,"(1315, 10)"
6,Transformed test set shape,"(564, 10)"
7,Numeric features,9
8,Preprocess,True
9,Imputation type,simple


In [16]:
# Compare models and allow pycaret to choose the best one
best_model = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.9817,0.9930,0.7518,0.9071,0.8182,0.8087,0.8148,0.0560
xgboost,Extreme Gradient Boosting,0.9795,0.9937,0.7375,0.8792,0.7960,0.7854,0.7920,0.0250
ada,Ada Boost Classifier,0.9764,0.9848,0.7679,0.8299,0.7870,0.7748,0.7812,0.0390
rf,Random Forest Classifier,0.9749,0.9833,0.6393,0.8399,0.7133,0.7015,0.7145,0.0790
et,Extra Trees Classifier,0.9688,0.9881,0.4750,0.9050,0.6056,0.5923,0.6333,0.0690
lr,Logistic Regression,0.9658,0.9830,0.5500,0.8056,0.6374,0.6205,0.6411,0.4960
dt,Decision Tree Classifier,0.9635,0.8157,0.6500,0.6952,0.6453,0.6275,0.6412,0.0130
lda,Linear Discriminant Analysis,0.9490,0.9531,0.0786,0.5000,0.1344,0.1288,0.1915,0.0130
ridge,Ridge Classifier,0.9445,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0090
dummy,Dummy Classifier,0.9445,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.3720


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

In [17]:
# Create model based on best model outputs
final_model = create_model(best_model)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9848,0.9931,0.7143,1.0000,0.8333,0.8256,0.8385
1,0.9773,0.9943,0.5714,1.0000,0.7273,0.7163,0.7470
2,0.9924,1.0000,0.8750,1.0000,0.9333,0.9293,0.9317
3,0.9773,0.9909,0.7500,0.8571,0.8000,0.7880,0.7900
4,0.9773,0.9970,0.7500,0.8571,0.8000,0.7880,0.7900
5,0.9924,1.0000,0.8571,1.0000,0.9231,0.9191,0.9221
6,0.9771,0.9931,0.7143,0.8333,0.7692,0.7573,0.7597
7,0.9924,0.9942,0.8571,1.0000,0.9231,0.9191,0.9221
8,0.9847,0.9954,0.8571,0.8571,0.8571,0.8491,0.8491


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [19]:
# Use model to create predictions within test set
predictions = predict_model(final_model, data=test)
predictions.head()

,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition,prediction_label,prediction_score
0,4,467,88.480003,4,8,4,0.014989,0.189465,0.571429,0,0.9999
1,3,1004,58.990002,7,6,1,0.026892,0.058755,0.259259,0,0.9950
2,1,162,14.000000,4,5,4,0.006173,0.086420,4.000000,0,0.9998
3,5,532,84.790001,9,3,4,0.026316,0.159380,0.642857,0,0.9999
4,5,890,42.160000,4,0,0,0.021348,0.047371,0.210526,0,0.9994


In [20]:
# Append prediction values to test dataset in a column named "fraud"
test['fraud'] = predictions['prediction_label']
test.head()

,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition,fraud
0,4,467,88.48,4,8,4,0.014989,0.189465,0.571429,0
1,3,1004,58.99,7,6,1,0.026892,0.058755,0.259259,0
2,1,162,14.00,4,5,4,0.006173,0.086420,4.000000,0
3,5,532,84.79,9,3,4,0.026316,0.159380,0.642857,0
4,5,890,42.16,4,0,0,0.021348,0.047371,0.210526,0


In [21]:
# Save the test dataset with predictions as a CSV file
test.to_csv('test_predictions.csv', index=False)

In [22]:
# Check accuracy manually
real = pd.read_csv('/Users/zarahaziz/Documents/Data_Science/All_Projects/6_FraudDetection/DMC-2019-realclass.csv', delimiter='|')
real.head()

,fraud
0,0
1,0
2,0
3,0
4,0


In [30]:
real.columns = ['real']
real.head()

,real
0,0
1,0
2,0
3,0
4,0


In [32]:
real['test'] = test['fraud']
real.head()

,real,test
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0


In [38]:
same_count = (real['real'] == real['test']).sum()
same_percentage = (same_count / len(real)) * 100
print(same_percentage)

# Approximately 97.25% of our predictions are accurate to those provided in the actual project solutions.

97.2544823446512


In [ ]:
differences = (real['real'] != real['test']).sum()
diff_percent = (differences / len(real)) *100
print(diff_percent)

# Approximately only 2.75% of our predictions were incorrect.